# But du notebook

Le but de ce notebook est de récupérer le texte des règles de donjons et dragons pour ensuite en faire un embedding que l'on pourra ensuite interroger.

## Import des bibliothèques

In [137]:
import numpy as np
import pandas as pd
from PyPDF2 import PdfReader
import re
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

## Récupération de notre pdf

In [ ]:
reader = PdfReader("Basic-Rules-FR.pdf")
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased")

## Extraction du texte des pages.

On ne récupère pas les pieds de pages car ils contiennent le numéro de la page et l'adresse du site.

In [ ]:
parts = []

# Fonction pour exclure les pieds de pages
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 500:
        parts.append(text)

for i in tqdm(range(len(reader.pages))):
    parts.append(reader.pages[i].extract_text()[37:])

In [ ]:
full_pdf_text = " ".join(parts)

## Embedding de notre texte

En vue d'interroger ce texte par un modèle de deep learning, il faut en faire un embedding. On pourrait le faire tout simplement par chunks de 1024 caractères mais on voudrait faire quelque chose de plus subtil.

Possibilités :
- Séparer par phrases.
- Mettre du recouvrement entre chunks.
- Regrouper les phrases par leur sémantique. (On faite un embedding des phrases et on fusionne les phrases qui possèdent un embedding proche).

Afin de concilier puissance et vitesse d'exécution, nous allons faire des groupes de plusieurs phrases (dans un maximum de 1024 caractères) avec une phrase de chevauchement.

**Note** : Afin d'améliorer notre séparation, nous aurions pu tenir compte des paragraphes. A faire éventuellement.

In [ ]:
# Mise en forme du document et suppression du sommaire
sentence_list = re.split('\\. |\\! |\\? ', full_pdf_text)
useful_sentences = sentence_list[343:]
for i in range(len(useful_sentences)):
    useful_sentences[i] = useful_sentences[i].replace("\n", " ")

In [ ]:
# On regroupe nos phrases dans une limite de 1024 caractères.
# On élimine les phrases trop longues car elles correspondent plutôt à des morceaux mal coupés (liste de sorts)

chunk_list = []
current_chunk = ""
threshold = 1024
i=0

useful_sentences = [element for element in useful_sentences if len(element) < 400]
while i < len(useful_sentences):
    accept_new_sentence = (len(current_chunk) + len(useful_sentences[i])) < threshold
    if accept_new_sentence:
        current_chunk = current_chunk + ". " + useful_sentences[i]
    else:
        chunk_list.append(current_chunk)
        current_chunk = ""
        if i > 0:
            current_chunk += useful_sentences[i-1] + ". " + useful_sentences[i]
        else:
            current_chunk += useful_sentences[i]
    i+=1

chunk_list.append(current_chunk)

In [165]:
# On fait l'embedding proprement dit.
embedding_data = embedding_model.encode(chunk_list)

In [167]:
embedding_dataframe = pd.DataFrame({"text_chunk" : chunk_list})
embedding_dataframe.to_csv("text_rag.csv", sep=";", index=False)
np.save("vector_rag.npy", embedding_data)